# Linear Regression

Refer to http://spark.apache.org/docs/latest/ml-features.html for information about how to transform data. 

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [7]:
data.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

In [8]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 
two columns needed, ("label", "feature")

In [9]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [21]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website","Length of Membership"],
    outputCol="features") 
# features are "Avg Session Length", "Time on App", "Time on Website",'Length of Membership'
output = assembler.transform(data)
output.select("features").head(1)

[Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [11]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

## Train Test Split

In [13]:
final_data = output.select("features",'Yearly Amount Spent')
train_data,test_data = final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean|  501.2961051169299|
| stddev|  81.06698968643927|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [14]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean| 494.41392852547426|
| stddev|  74.85468949078356|
|    min|  319.9288698031936|
|    max|  725.5848140556806|
+-------+-------------------+



## Fit Linear Regression Model

In [15]:
lr = LinearRegression(labelCol='Yearly Amount Spent')
lrModel = lr.fit(train_data)
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.92302306401339,38.4497785743354,0.565556565310133,61.43970203843536] Intercept: -1059.124708337252


In [16]:
test_results = lrModel.evaluate(test_data)

In [17]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  -11.2193767412856|
|  11.43007079268034|
| -2.948644265438986|
|  7.458807105067308|
|  4.506377170359087|
| -5.372941674344759|
| 3.9764871155747414|
| 0.5545083510176028|
|-3.2087750434934605|
| -8.915117379572791|
| -16.71209178844697|
|-13.522676893995822|
| 18.534765531418543|
| 17.611002220845023|
|-0.7250406470051303|
|  2.764175508806204|
| -4.323604408323774|
|  -9.01349803781153|
|  8.796404639668992|
| -2.117435066688813|
+-------------------+
only showing top 20 rows



In [22]:
test_results.r2

0.9828184315563212

In [18]:
unlabeled_data = test_data.select('features')
predictions = lrModel.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 331.1482465444792|
|[30.7377203726281...|450.35067140354954|
|[30.8794843441274...|493.15524425029366|
|[30.9716756438877...| 487.1798026518254|
|[31.0472221394875...| 387.9910220186623|
|[31.0613251567161...|492.92839973224636|
|[31.3662121671876...| 426.6123954409102|
|[31.3895854806643...| 409.5151027089653|
|[31.4252268808548...| 533.9754936982554|
|[31.5261978982398...| 418.0096435719106|
|[31.5702008293202...| 562.6575839298519|
|[31.5741380228732...| 557.9319490545827|
|[31.6005122003032...| 460.6380859596784|
|[31.6098395733896...|426.93454743026314|
|[31.6610498227460...|417.08339422690597|
|[31.7366356860502...| 494.1692707467257|
|[31.7656188210424...| 500.8776860439309|
|[31.8279790554652...|449.01624558475305|
|[31.8512531286083...| 464.1958420271294|
|[31.8530748017465...| 461.4025585290408|
+--------------------+------------

In [19]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.777710829809825
MSE: 95.60362907138034
